# 🤗 Hugging Face Datasets 완전정복

## 목차
1. [Datasets 라이브러리 소개](#1-datasets-라이브러리-소개)
2. [허브에서 데이터셋 로드하기](#2-허브에서-데이터셋-로드하기)
3. [데이터셋 생성하기](#3-데이터셋-생성하기)
4. [데이터셋 활용하기](#4-데이터셋-활용하기)
5. [데이터 전처리](#5-데이터-전처리)
6. [데이터셋 공유하기](#6-데이터셋-공유하기)

---

## 1. Datasets 라이브러리 소개

🤗 Datasets는 Hugging Face에서 제공하는 데이터셋 처리 라이브러리입니다.

### 주요 특징
- **메모리 효율성**: Apache Arrow 기반으로 대용량 데이터도 효율적으로 처리
- **빠른 처리 속도**: 최적화된 데이터 로딩 및 처리
- **다양한 형식 지원**: 텍스트, 이미지, 오디오, 비디오 등
- **스트리밍 지원**: 큰 데이터셋을 메모리에 모두 로드하지 않고 처리 가능
- **Hub 연동**: Hugging Face Hub의 수천 개 데이터셋에 쉽게 접근

### 설치 방법
```bash
pip install datasets
```


In [ ]:
# 필요한 라이브러리 설치 및 import
%pip install datasets transformers torch

import datasets
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names
from datasets import Dataset, DatasetDict, IterableDataset
import pandas as pd
import numpy as np

print(f"Datasets 버전: {datasets.__version__}")


## 2. 허브에서 데이터셋 로드하기

Hugging Face Hub에는 NLP, Computer Vision, Audio 등 다양한 분야의 데이터셋이 수천 개 제공됩니다.

### 2.1 데이터셋 정보 확인하기

데이터셋을 다운로드하기 전에 `load_dataset_builder()`를 사용해 정보를 미리 확인할 수 있습니다.


In [ ]:
# 데이터셋 정보 확인하기
ds_builder = load_dataset_builder("imdb")

print("=== 데이터셋 설명 ===")
print(ds_builder.info.description[:300] + "...")
print()

print("=== 데이터셋 특성(Features) ===")
print(ds_builder.info.features)
print()

print("=== 데이터셋 크기 ===")
print(f"총 크기: {ds_builder.info.dataset_size}")
print(f"다운로드 크기: {ds_builder.info.download_size}")


### 2.2 스플릿(Split) 확인하기

대부분의 데이터셋은 train, validation, test로 나뉘어 있습니다.


In [ ]:
# 데이터셋의 스플릿 확인
splits = get_dataset_split_names("imdb")
print("사용 가능한 스플릿:", splits)

# 특정 스플릿만 로드
train_dataset = load_dataset("imdb", split="train")
print(f"\n훈련 데이터셋: {train_dataset}")
print(f"데이터 개수: {len(train_dataset)}")

# 첫 번째 샘플 확인
print("\n=== 첫 번째 샘플 ===")
print(f"텍스트: {train_dataset[0]['text'][:200]}...")
print(f"라벨: {train_dataset[0]['label']}")


In [ ]:
# 전체 데이터셋 로드 (모든 스플릿)
full_dataset = load_dataset("imdb")
print("전체 데이터셋:")
print(full_dataset)
print()

# 각 스플릿별 데이터 개수 확인
for split_name, split_data in full_dataset.items():
    print(f"{split_name}: {len(split_data):,}개")


### 2.3 구성(Configuration) 확인하기

일부 데이터셋은 여러 하위 데이터셋(구성)을 포함합니다. 예: 다국어 데이터셋


In [ ]:
# 다국어 데이터셋 예제 - PolyAI/minds14
try:
    configs = get_dataset_config_names("PolyAI/minds14")
    print("PolyAI/minds14 데이터셋의 구성들:")
    print(configs[:10])  # 처음 10개만 출력
    
    # 특정 언어 구성 로드
    minds_en = load_dataset("PolyAI/minds14", "en-US", split="train[:100]")  # 처음 100개만
    print(f"\n영어(en-US) 구성: {minds_en}")
    print(f"첫 번째 샘플: {minds_en[0]}")
except Exception as e:
    print(f"에러 발생: {e}")
    print("다른 예제로 진행합니다...")


## 3. 데이터셋 생성하기

자신만의 데이터로 데이터셋을 생성하는 방법을 알아봅시다.

### 3.1 파일 기반 빌더

CSV, JSON, Parquet, TXT 등 다양한 파일 형식을 지원합니다.


In [ ]:
# 샘플 CSV 데이터 생성
import csv
import tempfile
import os

# 임시 CSV 파일 생성
temp_dir = tempfile.mkdtemp()
csv_file = os.path.join(temp_dir, "sample_data.csv")

# 샘플 데이터 작성
sample_data = [
    ["text", "label"],
    ["이 영화는 정말 재미있어요!", 1],
    ["별로였어요. 추천하지 않습니다.", 0],
    ["훌륭한 연출과 연기였습니다.", 1],
    ["지루하고 재미없었어요.", 0],
    ["최고의 영화 중 하나입니다!", 1]
]

with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(sample_data)

print(f"CSV 파일 생성: {csv_file}")

# CSV에서 데이터셋 생성
csv_dataset = load_dataset("csv", data_files=csv_file)
print("CSV 데이터셋:")
print(csv_dataset)
print("\n첫 번째 샘플:")
print(csv_dataset["train"][0])


In [ ]:
# JSON 파일에서 데이터셋 생성
import json

json_file = os.path.join(temp_dir, "sample_data.json")

# JSON 데이터 생성 (JSONL 형식)
json_data = [
    {"text": "멋진 영화였어요!", "label": 1, "rating": 4.5},
    {"text": "시간 낭비였습니다.", "label": 0, "rating": 2.0},
    {"text": "강력 추천합니다!", "label": 1, "rating": 5.0}
]

with open(json_file, 'w', encoding='utf-8') as f:
    for item in json_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"JSON 파일 생성: {json_file}")

# JSON에서 데이터셋 생성
json_dataset = load_dataset("json", data_files=json_file)
print("JSON 데이터셋:")
print(json_dataset)
print("\n샘플:")
for i in range(len(json_dataset["train"])):
    print(json_dataset["train"][i])


### 3.2 Python 딕셔너리에서 데이터셋 생성

메모리에 있는 Python 데이터로부터 직접 데이터셋을 생성할 수 있습니다.


In [ ]:
# from_dict() 방법
data_dict = {
    "pokemon": ["피카츄", "파이리", "꼬부기", "이상해씨"],
    "type": ["전기", "불", "물", "풀"],
    "power": [90, 85, 80, 82]
}

dict_dataset = Dataset.from_dict(data_dict)
print("딕셔너리에서 생성한 데이터셋:")
print(dict_dataset)
print("\n샘플:")
for i in range(len(dict_dataset)):
    print(dict_dataset[i])


In [ ]:
# from_generator() 방법 - 메모리 효율적
def data_generator():
    """대용량 데이터에 유용한 제네레이터"""
    for i in range(10):
        yield {
            "id": i,
            "text": f"이것은 {i}번째 텍스트입니다.",
            "score": i * 0.1
        }

# Dataset 생성
gen_dataset = Dataset.from_generator(data_generator)
print("제네레이터에서 생성한 데이터셋:")
print(gen_dataset)
print("\n처음 3개 샘플:")
for i in range(3):
    print(gen_dataset[i])

# IterableDataset 생성 (스트리밍용)
iterable_dataset = IterableDataset.from_generator(data_generator)
print("\n\n스트리밍 데이터셋:")
print(iterable_dataset)
print("스트리밍 샘플 (처음 3개):")
for i, sample in enumerate(iterable_dataset):
    if i >= 3:
        break
    print(sample)


### 3.3 폴더 기반 빌더 (ImageFolder & AudioFolder)

이미지나 오디오 데이터를 폴더 구조로 구성하여 데이터셋을 만들 수 있습니다.

**폴더 구조 예시:**
```
data/
├── train/
│   ├── cats/
│   │   ├── cat1.jpg
│   │   └── cat2.jpg
│   └── dogs/
│       ├── dog1.jpg
│       └── dog2.jpg
└── test/
    ├── cats/
    └── dogs/
```


In [ ]:
# 폴더 기반 데이터셋 예시 (실제로는 이미지 파일이 필요)
# dataset = load_dataset("imagefolder", data_dir="path/to/image/folder")

# 메타데이터와 함께 사용하는 예시
print("ImageFolder 사용 방법:")
print("""
# 이미지 데이터셋 로드
image_dataset = load_dataset("imagefolder", data_dir="path/to/images")

# 메타데이터가 있는 경우 (metadata.csv 파일 필요)
# metadata.csv 내용:
# file_name,caption
# cat1.jpg,귀여운 고양이
# dog1.jpg,활발한 강아지

image_dataset = load_dataset("imagefolder", data_dir="path/to/images")
""")

print("\nAudioFolder 사용 방법:")
print("""
# 오디오 데이터셋 로드
audio_dataset = load_dataset("audiofolder", data_dir="path/to/audio")
""")


## 4. 데이터셋 활용하기

로드한 데이터셋을 다양한 방법으로 조작하고 활용해봅시다.

### 4.1 기본 데이터셋 조작


In [ ]:
# 데이터셋 기본 조작 예제
dataset = load_dataset("imdb", split="train[:1000]")  # 처음 1000개만 로드

print("=== 데이터셋 정보 ===")
print(f"크기: {len(dataset)}")
print(f"컬럼: {dataset.column_names}")
print(f"특성: {dataset.features}")

# 인덱싱
print("\n=== 인덱싱 ===")
print("첫 번째 샘플:", dataset[0]["text"][:100] + "...")
print("처음 3개 라벨:", dataset[:3]["label"])

# 슬라이싱
print("\n=== 슬라이싱 ===")
subset = dataset[100:110]
print(f"100-110번째 샘플 개수: {len(subset['text'])}")

# 셔플
print("\n=== 셔플 ===")
shuffled = dataset.shuffle(seed=42)
print("셔플 전 첫 번째 라벨:", dataset[0]["label"])
print("셔플 후 첫 번째 라벨:", shuffled[0]["label"])


In [ ]:
# 필터링
print("=== 필터링 ===")
positive_reviews = dataset.filter(lambda example: example["label"] == 1)
print(f"긍정 리뷰 개수: {len(positive_reviews)}")

# 정렬
print("\n=== 정렬 ===")
sorted_by_length = dataset.sort("text", key=lambda x: len(x))
print("가장 짧은 리뷰:", sorted_by_length[0]["text"])
print("가장 긴 리뷰 길이:", len(sorted_by_length[-1]["text"]))

# 선택 (select)
print("\n=== 선택 ===")
sample_100 = dataset.select(range(100))
print(f"선택된 샘플 개수: {len(sample_100)}")

# 데이터셋 분할
print("\n=== 데이터셋 분할 ===")
train_test = dataset.train_test_split(test_size=0.2, seed=42)
print(f"훈련 데이터: {len(train_test['train'])}")
print(f"테스트 데이터: {len(train_test['test'])}")


## 5. 데이터 전처리

### 5.1 map() 함수를 이용한 데이터 변환

`map()` 함수는 데이터셋의 모든 요소에 함수를 적용하는 가장 중요한 메서드입니다.


In [ ]:
# 텍스트 전처리 함수 정의
def preprocess_text(examples):
    """텍스트를 소문자로 변환하고 길이 정보 추가"""
    return {
        "text_lower": [text.lower() for text in examples["text"]],
        "text_length": [len(text) for text in examples["text"]]
    }

# 작은 데이터셋으로 테스트
small_dataset = dataset.select(range(10))

# map 적용 (배치 처리)
processed = small_dataset.map(preprocess_text, batched=True)

print("=== 전처리 결과 ===")
for i in range(3):
    print(f"원본: {processed[i]['text'][:50]}...")
    print(f"소문자: {processed[i]['text_lower'][:50]}...")
    print(f"길이: {processed[i]['text_length']}")
    print("---")


### 5.2 토크나이저를 이용한 텍스트 토큰화


In [ ]:
# 토크나이저 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    """텍스트를 토큰화하는 함수"""
    return tokenizer(
        examples["text"], 
        padding="max_length",
        truncation=True,
        max_length=128
    )

# 토큰화 적용
tokenized_dataset = small_dataset.map(tokenize_function, batched=True)

print("=== 토큰화 결과 ===")
print("새로운 컬럼들:", tokenized_dataset.column_names)
print()

# 첫 번째 샘플 확인
sample = tokenized_dataset[0]
print("원본 텍스트:", sample["text"][:100] + "...")
print("토큰 ID 개수:", len(sample["input_ids"]))
print("토큰 ID (처음 10개):", sample["input_ids"][:10])
print("어텐션 마스크 (처음 10개):", sample["attention_mask"][:10])


### 5.3 PyTorch와 TensorFlow와의 호환성


In [ ]:
# PyTorch 형식으로 변환
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

print("=== PyTorch 형식 ===")
print("데이터 형식:", tokenized_dataset.format)
sample = tokenized_dataset[0]
print("input_ids 타입:", type(sample["input_ids"]))
print("input_ids 형태:", sample["input_ids"].shape)

# 원래 형식으로 되돌리기
tokenized_dataset.reset_format()
print("\n=== 원래 형식으로 복원 ===")
print("데이터 형식:", tokenized_dataset.format)

# 컬럼 제거/추가
print("\n=== 컬럼 조작 ===")
# 불필요한 컬럼 제거
clean_dataset = tokenized_dataset.remove_columns(["text"])
print("제거 후 컬럼:", clean_dataset.column_names)

# 새 컬럼 추가
def add_ids(examples):
    return {"id": list(range(len(examples["label"])))}

dataset_with_ids = clean_dataset.map(add_ids, batched=True)
print("추가 후 컬럼:", dataset_with_ids.column_names)


### 5.4 스트리밍 데이터셋

대용량 데이터셋은 스트리밍 방식으로 처리하여 메모리 사용량을 줄일 수 있습니다.


In [ ]:
# 스트리밍 데이터셋 로드
streaming_dataset = load_dataset("imdb", streaming=True)

print("=== 스트리밍 데이터셋 ===")
print("타입:", type(streaming_dataset["train"]))

# 스트리밍으로 데이터 처리
print("\n처음 5개 샘플 (스트리밍):")
for i, sample in enumerate(streaming_dataset["train"]):
    if i >= 5:
        break
    print(f"샘플 {i}: {sample['text'][:50]}... (라벨: {sample['label']})")

# 스트리밍 데이터셋에 map 적용
def simple_preprocess(example):
    example["text_length"] = len(example["text"])
    return example

processed_streaming = streaming_dataset["train"].map(simple_preprocess)

print("\n전처리된 스트리밍 데이터:")
for i, sample in enumerate(processed_streaming):
    if i >= 3:
        break
    print(f"길이: {sample['text_length']}, 텍스트: {sample['text'][:30]}...")


## 6. 데이터셋 공유하기

### 6.1 로컬에서 허브로 업로드

생성한 데이터셋을 Hugging Face Hub에 업로드하여 공유할 수 있습니다.


In [ ]:
# 데이터셋 허브에 업로드하는 방법 (실제로는 인증이 필요)

# 1. 로그인 (터미널에서 huggingface-cli login 실행)
print("데이터셋 업로드 방법:")
print("""
# 1. 허브 로그인
from huggingface_hub import login
login()  # 또는 터미널에서 huggingface-cli login

# 2. 데이터셋 업로드
my_dataset = Dataset.from_dict({
    "text": ["안녕하세요", "반갑습니다", "감사합니다"],
    "label": [0, 1, 0]
})

# 허브에 업로드
my_dataset.push_to_hub("username/my-dataset")

# 여러 스플릿이 있는 경우
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
dataset_dict.push_to_hub("username/my-dataset")
""")

# 로컬에 저장
print("\n로컬 저장 방법:")
sample_dataset = Dataset.from_dict({
    "text": ["샘플 텍스트 1", "샘플 텍스트 2"],
    "label": [0, 1]
})

# 로컬에 저장
save_path = os.path.join(temp_dir, "my_dataset")
sample_dataset.save_to_disk(save_path)
print(f"데이터셋 저장 완료: {save_path}")

# 로컬에서 로드
loaded_dataset = Dataset.load_from_disk(save_path)
print("로드된 데이터셋:", loaded_dataset)
print("샘플:", loaded_dataset[0])


### 6.2 데이터셋 카드 작성

데이터셋을 공유할 때는 `README.md` 파일에 데이터셋 카드를 작성하여 사용자들에게 정보를 제공해야 합니다.

**데이터셋 카드 예시:**
```markdown
---
license: apache-2.0
task_categories:
- text-classification
language:
- ko
---

# 한국어 감정 분석 데이터셋

## 데이터셋 설명
이 데이터셋은 한국어 리뷰 데이터의 감정을 분류하기 위한 데이터셋입니다.

## 데이터셋 구조
- **train**: 8,000개 샘플
- **test**: 2,000개 샘플

## 사용법
```python
from datasets import load_dataset
dataset = load_dataset("username/korean-sentiment")
```
```


## 7. 실습 과제

### 과제 1: 나만의 텍스트 분류 데이터셋 생성
1. 영화 리뷰 데이터 10개를 만들어 CSV 파일로 저장
2. 데이터셋으로 로드하고 train/test 분할 (8:2 비율)
3. 텍스트를 토큰화하고 PyTorch 형식으로 변환

### 과제 2: 데이터 전처리 파이프라인 구축
1. 텍스트 길이 정보 추가
2. 짧은 텍스트 필터링 (50자 이하 제거)
3. 결과를 로컬에 저장

### 과제 3: 스트리밍 데이터 처리
1. 큰 데이터셋을 스트리밍으로 로드
2. 배치 단위로 전처리 수행
3. 처음 100개 샘플만 추출하여 분석


In [ ]:
# 과제 1 해답 예시
print("=== 과제 1: 나만의 데이터셋 생성 ===")

# 1. 영화 리뷰 데이터 생성
movie_reviews = [
    ["이 영화는 정말 감동적이에요! 강력 추천합니다.", 1],
    ["지루하고 예측 가능한 스토리였어요.", 0],
    ["연출과 연기가 모두 훌륭했습니다.", 1],
    ["시간 낭비였어요. 별로 추천하지 않아요.", 0],
    ["최고의 영화 중 하나입니다!", 1],
    ["스토리가 너무 복잡해서 이해하기 어려웠어요.", 0],
    ["감동적인 결말이 인상 깊었습니다.", 1],
    ["액션 시퀀스가 정말 대단했어요!", 1],
    ["캐릭터들이 매력적이지 않았어요.", 0],
    ["다시 보고 싶은 영화입니다.", 1]
]

# CSV 파일로 저장
review_file = os.path.join(temp_dir, "movie_reviews.csv")
with open(review_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["review", "sentiment"])
    writer.writerows(movie_reviews)

# 2. 데이터셋 로드 및 분할
review_dataset = load_dataset("csv", data_files=review_file)["train"]
train_test = review_dataset.train_test_split(test_size=0.2, seed=42)

print(f"훈련 데이터: {len(train_test['train'])}개")
print(f"테스트 데이터: {len(train_test['test'])}개")

# 3. 토큰화 및 PyTorch 형식 변환
def tokenize_reviews(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=64)

tokenized_reviews = train_test.map(tokenize_reviews, batched=True)
tokenized_reviews.set_format("torch", columns=["input_ids", "attention_mask", "sentiment"])

print("토큰화 완료!")
print("훈련 데이터 첫 번째 샘플:")
print(f"Shape: {tokenized_reviews['train'][0]['input_ids'].shape}")
print(f"Label: {tokenized_reviews['train'][0]['sentiment']}")


In [ ]:
# 임시 파일들 정리
import shutil
shutil.rmtree(temp_dir)
print("임시 파일들이 정리되었습니다.")

print("\n🎉 Hugging Face Datasets 강의를 마치겠습니다!")
print("\n요약:")
print("✅ 허브에서 데이터셋 로드하기")
print("✅ 다양한 방법으로 데이터셋 생성하기") 
print("✅ 데이터 전처리 및 변환")
print("✅ 스트리밍 데이터셋 활용")
print("✅ 데이터셋 저장 및 공유")
print("\n추가 학습 자료:")
print("- 공식 문서: https://huggingface.co/docs/datasets")
print("- Hub 둘러보기: https://huggingface.co/datasets")
print("- 커뮤니티 포럼: https://discuss.huggingface.co")
